In [ ]:
import numpy as np
import pandas as pd
import random
import math,time,sys
from matplotlib import pyplot
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def sigmoid1(gamma):     #convert to probability
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))

def Vfunction1(gamma):
    return abs(np.tanh(gamma))

def Vfunction3(gamma):
    val = 1 + gamma*gamma
    val = math.sqrt(val)
    val = gamma/val
    return abs(val)

def fitness(position):
    cols=np.flatnonzero(position)
    val=1
    if np.shape(cols)[0]==0:
        return val
    clf=KNeighborsClassifier(n_neighbors=5)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)

    #in case of multi objective  []
    set_cnt=sum(position)
    set_cnt=set_cnt/np.shape(position)[0]
    val=omega*val+(1-omega)*set_cnt
    return val

def onecount(position):
    cnt=0
    for i in position:
        if i==1.0:
            cnt+=1
    return cnt

def allfit(population):
    x=np.shape(population)[0]
    acc=np.zeros(x)
    for i in range(x):
        acc[i]=fitness(population[i])     
        #print(acc[i])
    return acc

def initialize(popSize,dim):
    population=np.zeros((popSize,dim))
    minn = 1
    maxx = math.floor(0.8*dim)
    if maxx<minn:
        minn = maxx

    for i in range(popSize):
        random.seed(i**3 + 10 + time.time() ) 
        no = random.randint(minn,maxx)
        if no == 0:
            no = 1
        random.seed(time.time()+ 100)
        pos = random.sample(range(0,dim-1),no)
        for j in pos:
            population[i][j]=1
        # print(population[i])  
    return population


def toBinary(population,popSize,dimension,oldPop):

    for i in range(popSize):
        for j in range(dimension):
            temp = Vfunction3(population[i][j])

            if temp > 0.5: # vfunction
                population[i][j] = (1 - oldPop[i][j])
            else:
                population[i][j] = oldPop[i][j]
    return population

def selectParentRoulette(popSize,fitnList):
    fitnList = np.array(fitnList)
    fitnList = 1- fitnList/fitnList.sum()

    random.seed(time.time()+19)
    val = random.uniform(0,fitnList.sum())
    for i in range(popSize):
        if val <= fitnList[i]:
            return i
        val -= fitnList[i]
    return -1

In [3]:
## Parameters for GA
crossoverprob = 0.6
muprobmin = 0.01
muprobmax = 0.3

## Parameters for MRFA
omega = 0.85 #weightage for no of features and accuracy
popSize = 20
max_iter = 2
S = 2

In [4]:
df=pd.read_csv('dummy.csv')
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
dimension = np.shape(data)[1]

21 10


## GA

In [5]:
def GA(dataset,popSize,maxIter,randomstate):

    cross = 5
    test_size = (1/cross)
    trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size,random_state=randomstate) 

    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(trainX,trainy)
    val=clf.score(testX,testy)
    whole_accuracy = val

    population = initialize(popSize,dimension)
    GBESTSOL = np.zeros(np.shape(population[0]))
    GBESTFIT = 1000

    for currIter in range(1,maxIter):
        newpop = np.zeros((popSize,dimension))
        fitList = allfit(population)
        arr1inds = fitList.argsort()
        population = population[arr1inds]
        fitList= fitList[arr1inds]

        bestInx = np.argmin(fitList)
        fitBest = min(fitList)
        
        if fitBest<GBESTFIT:
            GBESTSOL = population[bestInx].copy()
            GBESTFIT = fitBest

        for selectioncount in range(int(popSize/2)):
            parent1 =   selectParentRoulette(popSize,fitList)
            parent2 = parent1
            while parent2 == parent1:
                random.seed(time.time())
                parent2 = selectParentRoulette(popSize,fitList)


            parent1 = population[parent1].copy()
            parent2 = population[parent2].copy()
            #cross over between parent1 and parent2
            child1 = parent1.copy()
            child2 = parent2.copy()
            for i in range(dimension):
                random.seed(time.time())
                if random.uniform(0,1)<crossoverprob:
                    child1[i]=parent2[i]
                    child2[i]=parent1[i]
            i = selectioncount
            j = int(i+(popSize/2))
            newpop[i]=child1.copy()
            newpop[j]=child2.copy()

        #mutation
        mutationprob = muprobmin + (muprobmax - muprobmin)*(currIter/maxIter)
        for index in range(popSize):
            for i in range(dimension):
                random.seed(time.time()+dimension+popSize)
                if random.uniform(0,1)<mutationprob:
                    newpop[index][i]= 1- newpop[index][i]
                    
        population = newpop.copy()
       
    return population

In [6]:
#GA(df,popSize,2,randomstate=0)

## Genetic Manta Ray

In [7]:
for train_iteration in range(1):

    cross = 5
    test_size = (1/cross)
    trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)


    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(trainX,trainy)
    val=clf.score(testX,testy)
    whole_accuracy = val

    #Initialize the pop
    population = initialize(popSize,dimension) # Initial population has been initialized(20*9)
    fitList = allfit(population)  ## taking the fitness of each of the 20 1-D array and storing it in a list
    fitBest = min(fitList)       ## taking the the min of (20,) 1-D array
    bestInx = np.argmin(fitList) ## taking the index of the min of (20,) 1-D array
    Mbest = population[bestInx].copy() ## taking the 1-D array with best fitness out of the 20 1-D array  
    
    for currIter in range(max_iter):

        popnew = GA(df,popSize,2,randomstate=0) ## initial pop = pop from GA
        for i in range(popSize):
            randNo = random.random() ## Based on this number either chain foraging or cyclone foraging will take place
            if randNo<0.5 :
                #chain foraging
                r = random.random()
                alpha = 2*r*(abs(math.log(r))**0.5)
                if i == 1:
                    popnew[i] = population[i] + r * (Mbest - population[i]) + alpha*(Mbest - population[i]) ## Eq(1)
                else:
                    popnew[i] = population[i] + r * (population[i-1] - population[i]) + alpha*(Mbest - population[i]) #Eq(2)
            else:
                #cyclone foraging
                cutOff = random.random()
                r = random.random()
                r1 = random.random()
                beta = 2 * math.exp(r1 * (max_iter - currIter + 1) / max_iter) * math.sin(2 * math.pi * r1) ## Eq(5)
                if (currIter/max_iter) < cutOff:  ## Stopping criteria for exploration/exploitation
                    # exploration
                    Mrand = np.zeros(np.shape(population[0]))
                    no = random.randint(1,max(int(0.1*dimension),2)) ## generates a random between 1 and (0.1*dimension)
                    pos = random.sample(range(0,dimension-1),no) ## returns a list with 'no' number of integers
                    for j in pos:
                        Mrand[j] = 1 ## converts the positions of those 0's to 1's that are returned in the 'pos' list

                    if (i==1) :
                        popnew[i] = Mrand + r * (Mrand - population[i]) + beta * (Mrand - population[i]) ## Eq(6)
                    else:
                        popnew[i] = Mrand + r * (population[i-1] - population[i]) + beta * (Mrand - population[i]) # Eq(7)
                else:
                    # exploitation
                    if (i == 1):
                        popnew[i] = Mbest + r * (Mbest - population[i]) + beta * (Mbest - population[i]) ## Eq(6)
                    else:
                        popnew[i] = Mbest + r * (population[i-1] - population[i]) + beta * (Mbest - population[i]) # Eq(7)

        
        popnew = toBinary(popnew,popSize,dimension,population) ## popnew after Manta Ray
        popnewTemp = popnew.copy()
        
        fitList = allfit(popnew)   ## taking the fitness of each of the 20 1-D array after Manta Ray and storing it in a list
        
        if min(fitList)<fitBest :  ## Checking whether fitness after manta ray has become better than fitness before manta ray
            fitBest = min(fitList)
            bestInx = np.argmin(fitList)
            Mbest = popnew[bestInx].copy()

        ## Somersault foraging
        for i in range(popSize):
            r2 = random.random()
            r3 = random.random()
            popnew[i] = popnew[i] + S * (r2*Mbest - r3*popnew[i]) ## Eq(8)

        popnew = toBinary(popnew,popSize,dimension,popnewTemp)
        #compute fitness for each individual
        fitList = allfit(popnew)
        if min(fitList)<fitBest :  ## To check whether Somersault better than (Chain+Cyclone)
            bestInx = np.argmin(fitList)
            fitBest = min(fitList)
            Mbest = popnew[bestInx].copy()

        population = popnew.copy()
            
    output = Mbest.copy()